# Партнерская задача: тематическая классификация текстов

In [3]:
!pip install emoji

  Using cached emoji-2.14.1-py3-none-any.whl.metadata (5.7 kB)
Using cached emoji-2.14.1-py3-none-any.whl (590 kB)


In [7]:
!pip install inflect

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import pymorphy2 - требуется лицензия MIT
import re

import emoji
import inflect
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avmar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\avmar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
pip install emoji inflect nltk

In [ ]:
pip freeze | grep -E "pandas|numpy|matplotlib|seaborn|scikit-learn|scipy" >> requirements.txt

In [ ]:
pip freeze | Select-String -Pattern "pandas|numpy|matplotlib|seaborn|scikit-learn|scipy" | Out-File -Append -FilePath requirements.txt

In [ ]:
pip freeze | findstr "pandas numpy matplotlib seaborn scikit-learn scipy" >> requirements.txt

## 1. Исследование датасета и предобработка данных

даны 6 датасетов (каждый датасет - отдельный класс) + описание

разделение "текст поста", video2text [OCR], speech2text [ASR]

In [107]:
df1 = pd.read_csv('https://github.com/martetten/Dataton_2/raw/main/data/raw/1.csv')
df2 = pd.read_csv('https://github.com/martetten/Dataton_2/raw/main/data/raw/2.csv')
df3 = pd.read_csv('https://github.com/martetten/Dataton_2/raw/main/data/raw/3.csv')
df4 = pd.read_csv('https://github.com/martetten/Dataton_2/raw/main/data/raw/4.csv')
df5 = pd.read_csv('https://github.com/martetten/Dataton_2/raw/main/data/raw/5.csv')
df6 = pd.read_csv('https://github.com/martetten/Dataton_2/raw/main/data/raw/6.csv')

In [108]:
print(df1.info())
print(df2.info())
print(df3.info())
print(df4.info())
print(df5.info())
print(df6.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   doc_text     1000 non-null   object
 1   image2text   568 non-null    object
 2   speech2text  338 non-null    object
dtypes: object(3)
memory usage: 23.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   doc_text     1000 non-null   object
 1   image2text   418 non-null    object
 2   speech2text  191 non-null    object
dtypes: object(3)
memory usage: 23.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   doc_text     1000 non-null   object
 1   image2text   521 non-null    object
 2   speech2text  2

In [ ]:
# df1 = df1.drop(columns='class')

In [148]:
df1['class'] = 1
df2['class'] = 2
df3['class'] = 3
df4['class'] = 4
df5['class'] = 5
df6['class'] = 6

In [178]:
df_gen = pd.concat([df1, df2, df3, df4, df5, df6]).reset_index()

In [150]:
df_gen.tail()

,index,doc_text,image2text,speech2text,class
5995,995,"Отказали ноги, стандартная ситуация! #щербако...",NaN,"Ночь, ожидания, холод, боль, словно я раскол, ...",6
5996,996,Осторожно пропала собака! #алексейщербаков #ще...,(папусов^ х,Всегда очень обидно читать подобные родные объ...,6
5997,997,Для ваших огонёчков🔥,v к видео,на телефонах огонёчки вдруг зажигает полный за...,6
5998,998,#зиминарулит 😂😂😂 #актрисатеатраикино #виктори...,о 6,"Что вы умеете делать? Всё. Стрелять, варить ха...",6
5999,999,"Девочки, ну мы?! @katiashmatia правильно пост...",NaN,милая он не отвечает мне на сообщения уже четы...,6


- очистка от символов, ссылок, смайликов
- удаление повторов
- преобразование OneHotEncoding?

In [112]:
df_gen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        6000 non-null   int64 
 1   doc_text     6000 non-null   object
 2   image2text   3420 non-null   object
 3   speech2text  1871 non-null   object
 4   class        6000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.5+ KB


Функция для преобразования эмоджи в их текстовые описания и очищает получившийся текст

In [179]:
def extract_emojis(text):
    """Извлекает эмодзи из текста и возвращает их текстовые описания через запятую"""
    if not isinstance(text, str):
        return np.nan

    # Находим все эмодзи в тексте
    emoji_chars = [c for c in text if c in emoji.EMOJI_DATA]

    if not emoji_chars:
        return np.nan

    # Преобразуем эмодзи в текстовые описания (без : и _)
    emoji_descriptions = [
        emoji.demojize(e, delimiters=(" ", " "))
        .replace(":", "").replace("_", " ")
        .strip()
        for e in emoji_chars
    ]

    # Объединяем через запятую
    return ", ".join(emoji_descriptions)

In [180]:
def remove_emojis(text):
    """Удаляет все эмодзи из текста"""
    if not isinstance(text, str):
        return np.nan
    return emoji.replace_emoji(text, replace="")

Функция для предварительной очистки и нормализации текста

In [181]:
def base_clean_text(text: str):
    if not isinstance(text, str):
        return np.nan

    text = text.lower()

    # Удаляем URL, хэштеги, упоминания ДО split()
    text = re.sub(r'(https?://\S+)|(#\w+)|(@\w+)', '', text)

    # Удаляем скобки/кавычки, мешающие обработке
    text = re.sub(r'[()\[\]{}"\']', '', text)

    # Оставляем только буквы и пробелы
    text = re.sub(r'[^\sа-яёa-z]', '', text)

    # Убираем повторы букв (например, "привееет" -> "привет")
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    stop_words = set(stopwords.words('russian'))
    temp = inflect.engine()
    words = []

    for word in text.split():
        if word.isdigit():
            words.append(temp.number_to_words(word))
        elif word and word not in stop_words:  # игнорируем пустые строки и стоп-слова
            words.append(word)

    return ' '.join(words) if words else np.nan

Функция для приведения слов к их корневой форме

In [182]:
def stem_russian_text(text):
    stemmer = RussianStemmer()
    words = text.split()
    stemmed_words = []
    for word in words:
        if re.match('[а-яА-Я]', word):
            stemmed_word = stemmer.stem(word)
            stemmed_words.append(stemmed_word)
        else:
            stemmed_words.append(word)
    return ' '.join(stemmed_words)

In [183]:
df_gen['emojis_doc_text'] = df_gen['doc_text'].apply(extract_emojis)

In [163]:
df_gen['emojis_doc_text']

,emojis_doc_text
0,"no one under eighteen, smiling cat with heart-..."
1,"star, star, star"
2,NaN
3,flexed biceps
4,NaN
...,...
5995,NaN
5996,NaN
5997,fire
5998,"face with tears of joy, face with tears of joy..."


In [184]:
df_gen['doc_text'] = df_gen['doc_text'].apply(remove_emojis)

In [185]:
df_gen['doc_text']

,doc_text
0,Твой лучший секс спрятан здесь Делюсь канало...
1,Кнопка: START(https://t.me/major/start?start...
2,А продолжение где? Правильно. В моем сообществ...
3,Тем временем моя авторская телега уверенно в т...
4,"У меня есть двоюродная сестра, у нее есть сын ..."
...,...
5995,"Отказали ноги, стандартная ситуация! #щербако..."
5996,Осторожно пропала собака! #алексейщербаков #ще...
5997,Для ваших огонёчков
5998,#зиминарулит #актрисатеатраикино #викторията...


In [191]:
df_gen.iloc[1]['cleaned_doc_text']

'кнопка start'

In [186]:
for col in ['doc_text', 'image2text', 'speech2text']:
    df_gen[f'cleaned_{col}'] = [base_clean_text(x) if isinstance(x, str) else np.nan
                                 for x in df_gen[col]]

In [189]:
df_gen

,index,doc_text,image2text,speech2text,class,emojis_doc_text,cleaned_doc_text,cleaned_image2text,cleaned_speech2text
0,0,Твой лучший секс спрятан здесь Делюсь канало...,NaN,NaN,1,"no one under eighteen, smiling cat with heart-...",твой лучший секс спрятан делюсь каналом диплом...,NaN,NaN
1,1,Кнопка: START(https://t.me/major/start?start...,NaN,NaN,1,"star, star, star",кнопка start,NaN,NaN
2,2,А продолжение где? Правильно. В моем сообществ...,девчонки не умеют переписываться sanille они э...,NaN,1,NaN,продолжение правильно моем сообществе вк ссылк...,девчонки умеют переписываться sanille это очен...,NaN
3,3,Тем временем моя авторская телега уверенно в т...,10:42 nuil пул telegram ^ 51 142 подписчика 12...,NaN,1,flexed biceps,временем авторская телега уверенно тройке силь...,nuil пул telegram подписчика авторские маргари...,NaN
4,4,"У меня есть двоюродная сестра, у нее есть сын ...","т с не ^ е z 8 * \""8 ничего не знала о стендап...",NaN,1,NaN,двоюродная сестра сын антон двоюродный племянн...,т е z знала стендапе маргарита родина,NaN
...,...,...,...,...,...,...,...,...,...
5995,995,"Отказали ноги, стандартная ситуация! #щербако...",NaN,"Ночь, ожидания, холод, боль, словно я раскол, ...",6,NaN,отказали ноги стандартная ситуация,NaN,ночь ожидания холод боль словно раскол вижу не...
5996,996,Осторожно пропала собака! #алексейщербаков #ще...,(папусов^ х,Всегда очень обидно читать подобные родные объ...,6,NaN,осторожно пропала собака,папусов х,очень обидно читать подобные родные объявления...
5997,997,Для ваших огонёчков,v к видео,на телефонах огонёчки вдруг зажигает полный за...,6,fire,ваших огонёчков,v видео,телефонах огонёчки зажигает полный зал телефон...
5998,998,#зиминарулит #актрисатеатраикино #викторията...,о 6,"Что вы умеете делать? Всё. Стрелять, варить ха...",6,"face with tears of joy, face with tears of joy...",правда умею всё нашла видео,NaN,умеете делать всё стрелять варить халву подков...


In [ ]:
df_gen.to_csv('df_gen.csv', index=False)

In [ ]:
# Замена пропусков в источниках (стоит ли делать это сейчас?)
# df_gen['image2text'] = df_gen['image2text'].fillna('')
# df_gen['speech2text'] = df_gen['speech2text'].fillna('')

In [ ]:
# Удаление полностью пустых постов (таких на самом деле нет)
df_gen = df_gen[~(df_gen['doc_text'].str.len() + df_gen['image2text'].str.len() + df_gen['speech2text'].str.len() == 0)]

In [ ]:
# morph = pymorphy2.MorphAnalyzer() (вопрос согласия партнера на использование лицензии MIT)
# def preprocess(text):
#    text = re.sub(r'[^\w\s]', '', text.lower())  # Удаление пунктуации
#    words = [morph.parse(word)[0].normal_form for word in text.split()]  # Лемматизация
#    return ' '.join(words)

In [ ]:
import re
from nltk.corpus import stopwords

STOPWORDS_RU = set(stopwords.words('russian')) | {'это', 'всё', 'весь'} # (что ещё добавить?)

def clean_text(text):
    # Удаление URL, хэштегов, упоминаний
    text = re.sub(r'(http\S+)|(#\w+)|(@\w+)', '', text)

    # Нормализация повторов: "привеееет" → "привет"
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # Удаление редких символов (сохраняем кириллицу и основные знаки)
    text = re.sub(r'[^а-яё\s!?.,:;-]', '', text, flags=re.IGNORECASE)

    return text.strip()

Обработка выбросов
- Слишком короткие тексты (менее 2 слов после очистки)
- Повторы: дубликаты с одинаковым user_text
- Аномалии OCR/ASR: тексты с >50% не-буквенных символов

In [ ]:
# Удаление слишком коротких текстов
df['num_words'] = df['text'].apply(lambda x: len(x.split()))
df = df[df['num_words'].between(2, 30)]

# Поиск аномалий OCR/ASR
def is_low_quality(text):
    char_ratio = sum(c.isalpha() for c in text) / len(text) if text else 0
    return char_ratio < 0.4

mask = df[['ocr_text', 'asr_text']].applymap(is_low_quality)
df['ocr_quality'] = ~mask['ocr_text']
df['asr_quality'] = ~mask['asr_text']

## 2. Feature Engineering

Попробовать кластеризацию, насколько реально выполнена разметка/разделение датасетов

3 версии датасета:
- только посты
- посты + дополнения
- все источники с маркерами [post] text, [OCR] text, [ASR] text

In [ ]:
# взвешенная конкатенация (тект + OCR + ASR)
def combine_sources(row):
    main_text = row['user_text']
    supplements = ' '.join([row['ocr_text'], row['asr_text']])
    return f"{main_text} [SUPP] {supplements}"  # Маркер для дополнений

## 3.  Подбор признаков, их анализ и оценка важности

## 4. Обучение нескольких моделей, их сравнение

## 5. Подбор гиперпараметров

## 6. Выбор лучшей модели и объяснение выбора

## 7. Предсказание на тестовых данных